In [ ]:
import os
import numpy as np
import tensorflow as tf

In [ ]:
# Load the TFLite model and allocate tensors.
# Let's load the TFLite model we saved to "./saved_tflite_models/model_2.tflite" above.
interpreter = tf.lite.Interpreter(model_path="./saved_tflite_models/model_2.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)

# "set_tensor" sets the data at the model's input.
interpreter.set_tensor(input_details[0]['index'], input_data)

# "invoke()" sends the data through the model.
interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
# Let's get the output from the model and print it.
output_data = interpreter.get_tensor(output_details[0]['index'])